**Transfer Learning via Xception model (Dogs vs. Cats)**

* Part 1 [Intro to CNN (Dogs vs. Cats)](https://www.kaggle.com/imcr00z/intro-to-cnn-dogs-vs-cats)
* Part 2 [Intro to CNN: Augmentation & Dropout](https://www.kaggle.com/imcr00z/intro-to-cnn-augmentation-dropout)
* Part 3 [Transfer Learning (Dogs vs. Cats) 98% acc.](https://www.kaggle.com/imcr00z/transfer-learning-dogs-vs-cats-98-acc)

In this part i use transfer learning and pretrained models:
* Xception (from Franc¸ois Chollet). Original document [here](https://arxiv.org/pdf/1610.02357.pdf)
* MobileNet via Google. Original document [here](https://arxiv.org/pdf/1704.04861.pdf)

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import shutil
import tqdm

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load data

In [ ]:
CONTENT_DIR = '/kaggle/content'
TRAIN_DIR = CONTENT_DIR + '/train'
VALID_DIR = CONTENT_DIR + '/valid'

if not os.path.exists(CONTENT_DIR):
    # Extract dataset
    import zipfile
    with zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r') as zipf:
        zipf.extractall(CONTENT_DIR)

    # Split cats and dogs images to train and valid datasets
    img_filenames = os.listdir(TRAIN_DIR)
    dog_filenames = [fn for fn in img_filenames if fn.startswith('dog')]
    cat_filenames = [fn for fn in img_filenames if fn.startswith('cat')]
    dataset_filenames = train_test_split(
        dog_filenames, cat_filenames, test_size=0.1, shuffle=True, random_state=42
    )

    # Move images
    make_dirs = [d + a for a in ['/dog', '/cat'] for d in [TRAIN_DIR, VALID_DIR]]
    for dir, fns in zip(make_dirs, dataset_filenames):
        os.makedirs(dir, exist_ok=True)
        for fn in tqdm.tqdm(fns):
            shutil.move(os.path.join(TRAIN_DIR, fn), dir)
        print('elements in {}: {}'.format(dir, len(os.listdir(dir))))

In [ ]:
BATCH_SIZE = 32
IMAGE_SHAPE = 128

# Preprocessing

In [ ]:
# make data generators
train_generator = ImageDataGenerator(rescale=1./255)
valid_generator = ImageDataGenerator(rescale=1./255)
train_data = train_generator.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)
valid_data = valid_generator.flow_from_directory(
    directory=VALID_DIR,
    target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Xception
In this part, I extract 'bottleneck features' from an Xception model without fully connected layers and train a new fully connected model on them.
1. load and train xception, get bottleneck features
2. create model with dense layers
3. train dense layers on bottleneck features

In [ ]:
# load xception model
xception_model = tf.keras.applications.xception.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(IMAGE_SHAPE, IMAGE_SHAPE, 3),
    pooling='avg'
)

In [ ]:
# look at this, it's very scary :)
# change include_top=True -> last layer added
# tf.keras.utils.plot_model(xception_model, dpi=48, show_shapes=True)

In [ ]:
# bottleneck features for train dataset
train_bottleneck = xception_model.predict_generator(
    train_data, train_data.n // BATCH_SIZE, verbose=1
)
# np.save(open('train_bottleneck.np', 'wb'), train_bottleneck)

In [ ]:
# bottleneck features for valid dataset
valid_bottleneck = xception_model.predict_generator(
    valid_data, valid_data.n // BATCH_SIZE, verbose=1
)
# np.save(open('valid_bottleneck.np', 'wb'), valid_bottleneck)

In [ ]:
# create simple model for classification
model = tf.keras.models.Sequential([
    Dense(units=256, activation='relu', input_shape=xception_model.output_shape[1:]),
    Dropout(0.5),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# get predicted features and classify it
EPOCHS = 10
history = model.fit(
    x=train_bottleneck,
    y=train_data.labels[:len(train_bottleneck)],
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(valid_bottleneck, valid_data.labels[:len(valid_bottleneck)])
)

Look to validation loss and accuracy.

In [ ]:
def show_graphs(history):
    plt.figure(figsize=(12, 8))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='valid')
    plt.legend(loc='lower right')
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='valid')
    plt.legend(loc='upper left')
    plt.title('Loss (sparse_categorical_crossentropy)')

    plt.show()
    
show_graphs(history)

# MobileNet
In this case i use classical path with frozen layers.

In [ ]:
mobilenet_model = tf.keras.applications.mobilenet.MobileNet(
    include_top=False,
    weights='imagenet',
    input_shape=(IMAGE_SHAPE, IMAGE_SHAPE, 3),
    pooling='avg'
)
mobilenet_model.trainable = False # freeze convolutional layers

In [ ]:
# new fully connected part of the network
dense_model = tf.keras.models.Sequential([
    Dense(units=1000, activation='relu'),
    Dropout(0.5),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=2, activation='softmax')
])

In [ ]:
# build a new model
model2 = tf.keras.models.Sequential([
    mobilenet_model,
    dense_model
])

In [ ]:
model2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10
train_data.reset()
valid_data.reset()
history = model2.fit_generator(
    train_data,
    steps_per_epoch=train_data.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=valid_data,
    validation_steps=valid_data.n // BATCH_SIZE
)

In [ ]:
show_graphs(history)

# MobileNet with native output layer

Only for fun :)

**Achtung! Attention! Vnimanie!** If your datasets don't match very well, that's a bad idea!

In [ ]:
IMAGE_SHAPE = 224
example_data = train_generator.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMAGE_SHAPE, IMAGE_SHAPE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)
example_x, example_y = example_data.next()
example_classes = list(example_data.class_indices.keys())
example_y_classes = [example_classes[int(i)] for i in example_y]

In [ ]:
mobilenet_native = tf.keras.applications.mobilenet.MobileNet(
    include_top=True,
    weights='imagenet',
    input_shape=(IMAGE_SHAPE, IMAGE_SHAPE, 3),
    pooling='avg'
)
mobilenet_native.compile()
example_pred = mobilenet_native.predict(
    example_x
)

In [ ]:
labels_path = tf.keras.utils.get_file(
    'ImageNetLabels.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
)
imagenet_labels = np.array(open(labels_path).read().splitlines())
result = imagenet_labels[np.argmax(example_pred, axis=1)]

In [ ]:
NUM_ROWS = 5
NUM_COLS = 5
NUM_IMAGES = NUM_COLS * NUM_ROWS
plt.figure(figsize=(2*NUM_COLS, 2*NUM_ROWS))
for i in range(NUM_IMAGES):
    plt.subplot(NUM_ROWS, NUM_COLS, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(example_x[i], cmap=plt.cm.binary)
    plt.xlabel('{} {:.0%}\n({})'.format(result[i], np.max(example_pred[i]), example_y_classes[i]))
plt.tight_layout()
plt.show()